# DETERMINE R function of $\theta$

NOTA: t in ms

In [13]:
from labbiofisica import Interpolazione
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import os
import plotly.graph_objects as go 

In [2]:
# Carico i dati {θ: {x: [], y: []}}
# NOTA: sigmay = 0.001 per tutti i dati
data = {}

TETA = np.array([50,60,70,80,90,100,110,120,130])
GAMMA = []

for θ in TETA:
    filename = f'./data/g_{θ}.txt'
    read = pd.read_csv(filename,sep='\t ',header=None,engine='python')
    read = read[(read[0] >= 0.1) & (read[0] <= 10)]
    data[θ] = {'x':read[0].to_numpy(),'y':read[1].to_numpy(),'sigmay': np.float64(0.001)}


In [3]:
# plot row data

fig = go.Figure()

# fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='data'))

fig.update_layout(
	xaxis_type="log",
	yaxis_title="g<sup>2</sup>(τ)",
	xaxis_title="t (ms)",
	title='g<sup>2</sup>(τ) vs t',
	xaxis=dict(range=[np.log10(0.1), np.log10(10)]),
	# yaxis=dict(range=[0, 0.06])
)

for θ in data.keys():
	fig.add_trace(go.Scatter(x=data[θ]['x'], y=data[θ]['y'], mode='markers', name=f'θ={θ}°',error_y=dict( 
			type='constant', 
			value=0.001, 
			color='black',
			thickness=1.5, 
			width=3, 
		)))

fig.show()

In [4]:
# fit data

f = lambda x, A, B, γ: A*np.exp(-γ*x) + B

for key in data.keys():
    x = data[key]['x']
    y = data[key]['y']
    sigmay = data[key]['sigmay']
    m = Interpolazione(x, y, sigmay, f, [0.05, 0.001, 2.7],['A','B','γ'])
    data[key]['fit'] = m
    # print(m.values['γ'])
    data[key]['gamma'] = m.values['γ']
    GAMMA.append(m.values['γ'])

GAMMA = np.array(GAMMA) 

# PLOT FIT

In [21]:
colors = px.colors.qualitative.D3

fig = go.Figure()
for i,θ in enumerate(data.keys()):
    x = data[θ]['x']
    y = data[θ]['y']
    γ = data[θ]['gamma']
    sigmay = data[θ]['sigmay']
    X,Y = data[θ]['fit'].draw()

    # plt.plot(X,Y)
    # plt.xlim(0,1)
    # plt.show()

    fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=colors[i], showlegend=False))

    fig.add_trace(go.Scatter(x=data[θ]['x'], y=data[θ]['y'], mode='markers', name=f'θ={θ}°',
        marker=dict(color = colors[i]),
        error_y=dict( 
			type='constant', 
			value=0.001, 
			color=colors[i],
			thickness=1.5, 
			width=3, 
		)))

fig.update_layout(
        xaxis_type="log",
        yaxis_title="g<sup>2</sup>(τ)",
        xaxis_title="t (ms)",
        title='g<sup>2</sup>(τ) vs t',
        xaxis=dict(range=[np.log10(0.1), np.log10(10)]),
        width=800,
        height=600,
        # yaxis=dict(range=[0, 0.06])
    )
fig.show()